## Convert *existing* paradigm (PsychToolbox input) into FreeSurfer-friendly .par files

In [1]:
import os
import pandas as pd
import numpy as np

par_path='/Users/emilyhahn/projects/msit_BSM/params'

df = pd.read_csv(os.path.join(par_path,'msit_I-C.par'),index_col=False,sep='\t',\
                  names=['Time','Condition','Duration','Slope','Cond_IC'])

# display(df)

Condition=[]
# StimOnset=[]
# ResponseOnset=[]
Slope=np.zeros(len(df))

for i in df.Condition:
    Condition.append(i)

df2_data={'Condition': Condition}
df2=pd.DataFrame(df2_data)

# display(df2)

df2.to_csv(os.path.join(par_path,'MSIT_par_reformat.csv'),index=False)

## Configure files to be FreeSurfer Compatible

In [2]:
import os
import pandas as pd
import numpy as np

bdir='/Users/emilyhahn/projects/msit_modeling/msit_mri_behav'
indiv_par_path='/Users/emilyhahn/projects/msit_modeling/msit_indiv_pars'
par_path='/Users/emilyhahn/projects/msit_BSM/params'
msit_path='/Users/emilyhahn/projects/msit_modeling'

dfo = os.path.join(msit_path,'msit_mri_behav.txt')

subjects=[]
orig_conds=[]

for i in df2['Condition']:
    orig_conds.append(i)

f = open(dfo, 'r')
f=f.readlines()
for line in f:
    subjects.append(line.strip())
    
for subject in subjects:
    onsets=[]
    conditions=[]
    response_onset=[]
    stim_response_onsets=[]
    csv=pd.read_csv(os.path.join(bdir,'%s' % subject))
    df=pd.DataFrame(csv)
    for stim_onset,condition,res_onset in zip(df['StimOnset'],df['Condition'],df['ResponseOnset']):
        condition=str(condition)
        if '1' in condition or '2' in condition:
            stim_response_onsets.append('%.2f:%.2f' % (stim_onset,1.75))
#             stim_response_onsets.append('%.2f:%.2f' % (stim_onset,(res_onset-stim_onset)))
            onsets.append(stim_onset)
            conditions.append(condition)
            response_onset.append(res_onset)
        elif condition=='0':
            continue
        else:
            print("CHECK SUBJECT: %s" % subject)
    run1=os.path.join(par_path,'%s.csv' % 'MSIT_par_reformat')
    df1=pd.read_csv(run1)
    df1=pd.DataFrame(df1)
    df1['Condition'] = conditions
    df1['StimOnset'] = onsets
    df1['ResponseOnset'] = response_onset
    df1['StimResponseOnset'] = stim_response_onsets
    
#     df1=df1.drop(df1.index[-1])
#     df1=df1.drop(df1.index[0:2])
            
    df1['StimResponseOnset'].to_csv(os.path.join(indiv_par_path,'%s_durations_dmBLOCK.par'% subject[:5].lower()),\
                   sep='\t',index=False,header=None)
#     df1['StimResponseOnset'].to_csv(os.path.join(indiv_par_path,'%s_durations_dmBLOCK.par'% subject[:5].lower()),\
#                    sep='\t',index=False,header=None)

/Users/emilyhahn/anaconda2/envs/ipykernel_py3/lib/python3.7/site-packages/ipykernel_launcher.py:54: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
